In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error


In [26]:
items_category=pd.read_csv("data/item_categories.csv")
items=pd.read_csv("data/items.csv")
sales=pd.read_csv("data/sales_train.csv")
shops=pd.read_csv("data/shops.csv")
test=pd.read_csv("data/test.csv")
sample_submission=pd.read_csv("data/sample_submission.csv")

In [9]:
pre_df = sales.copy()
pre_df = pre_df.pivot_table(
    index=['shop_id', 'item_id'],
    values=['item_cnt_day'],
    columns=['date_block_num'],
    fill_value=0,
    aggfunc='sum'
).reset_index()

pre_df

shop_id item_id item_cnt_day                        ...        \
date_block_num                            0   1  2  3  4  5  6  7  ... 24 25   
0                    0      30            0  31  0  0  0  0  0  0  ...  0  0   
1                    0      31            0  11  0  0  0  0  0  0  ...  0  0   
2                    0      32            6  10  0  0  0  0  0  0  ...  0  0   
3                    0      33            3   3  0  0  0  0  0  0  ...  0  0   
4                    0      35            1  14  0  0  0  0  0  0  ...  0  0   
...                ...     ...          ...  .. .. .. .. .. .. ..  ... .. ..   
424119              59   22154            1   0  0  0  0  0  0  0  ...  0  0   
424120              59   22155            0   0  0  0  0  0  1  0  ...  0  0   
424121              59   22162            0   0  0  0  0  0  0  0  ...  0  9   
424122              59   22164            0   0  0  0  0  0  0  0  ...  0  2   
424123              59   22167            0   0  0  0  0  0  0  0  ...  0  0   

                                        
date_block_num 26 27 28 29 30 31 32 33  
0               0  0  0  0  0  0  0  0  
1               0  0  0  0  0  0  0  0  
2               0  0  0  0  0  0  0  0  
3               0  0  0  0  0  0  0  0  
4               0  0  0  0  0  0  0  0  
...            .. .. .. .. .. .. .. ..  
424119          0  0  0  0  0  0  0  0  
424120          0  0  0  0  0  0  0  0  
424121          4  1  1  0  0  1  0  0  
424122          1  2  0  0  1  0  0  0  
424123          0  0  0  0  0  0  0  0  

[424124 rows x 36 columns]

In [12]:
full_train_df = test.copy()
full_train_df = full_train_df.merge(pre_df, how='left', on=['shop_id', 'item_id']).fillna(0).drop(
    ['ID', 'shop_id', 'item_id'], axis=1)

full_train_df

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X_train, y_train = full_train_df.values[:,:-2], full_train_df.values[:, -2:-1].ravel()
X_valid, y_valid = full_train_df.values[:,1:-1], full_train_df.values[:, -1:].ravel()
X_test = full_train_df.values[:, 2:]

X_train


array([[0., 0., 0., ..., 1., 1., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
tf.keras.backend.clear_session()

lstm_model = tf.keras.Sequential([
    tf.keras.layers.Reshape(input_shape=(32,), target_shape=(32, 1,)),
    tf.keras.layers.LSTM(units=32, input_shape=(32, 1)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1)
])

lstm_model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(0.1),
    metrics=['mse']
)

lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 1)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [17]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=10)

lstm_model.fit(X_train, y_train, batch_size=4096, epochs=30,
          validation_data=(X_valid, y_valid),
          callbacks=[early_stop])

Epoch 1/30
53/53 [==============================] - 15s 280ms/step - loss: 6.3521 - mse: 6.3521 - val_loss: 30.3249 - val_mse: 30.3249
Epoch 2/30
53/53 [==============================] - 14s 261ms/step - loss: 5.7448 - mse: 5.7448 - val_loss: 29.9688 - val_mse: 29.9688
Epoch 3/30
53/53 [==============================] - 14s 268ms/step - loss: 5.5854 - mse: 5.5854 - val_loss: 29.8318 - val_mse: 29.8318
Epoch 4/30
53/53 [==============================] - 13s 253ms/step - loss: 5.6154 - mse: 5.6154 - val_loss: 29.8097 - val_mse: 29.8097
Epoch 5/30
53/53 [==============================] - 14s 259ms/step - loss: 5.4421 - mse: 5.4421 - val_loss: 29.5768 - val_mse: 29.5768
Epoch 6/30
53/53 [==============================] - 16s 294ms/step - loss: 5.2147 - mse: 5.2147 - val_loss: 29.4258 - val_mse: 29.4258
Epoch 7/30
53/53 [==============================] - 13s 248ms/step - loss: 5.3291 - mse: 5.3291 - val_loss: 29.7414 - val_mse: 29.7414
Epoch 8/30
53/53 [==============================] - 13s

In [22]:
y_pred = lstm_model.predict(X_valid)
print('LSTM RMSE =', mean_squared_error(y_valid, y_pred, squared=False))


LSTM RMSE = 5.462556809921868


In [31]:
from xgboost import XGBRegressor

RANDOM_STATE = 42
xgb_model = XGBRegressor(
    # learning_rate=0.05,
    max_depth=16,
    n_estimators=200,
    seed=RANDOM_STATE,
)

xgb_model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="rmse",
              eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=10)

y_pred = xgb_model.predict(X_valid)
print('XGBoost RMSE =', mean_squared_error(y_valid, y_pred, squared=False))
submission=sample_submission.copy()
submission['item_cnt_month'] = xgb_model.predict(X_test)
submission.to_csv("submission_XGBST.csv",index=False)



[0]	validation_0-rmse:2.06742	validation_1-rmse:5.33777
[10]	validation_0-rmse:1.20547	validation_1-rmse:4.99761
[20]	validation_0-rmse:1.18511	validation_1-rmse:4.96881
[30]	validation_0-rmse:1.17975	validation_1-rmse:4.96471
[39]	validation_0-rmse:1.17437	validation_1-rmse:4.96533
XGBoost RMSE = 4.964799001701483


In [27]:
submission=sample_submission.copy()
submission["item_cnt_month"]=lstm_model.predict(X_test)

In [28]:
submission.to_csv("submission_LSTM.csv",index=False)